# Task 1.3 — Hooke–Jeeves and GPS (Minimal)
*Libraries*: only `numpy`, `math`, `time`, `matplotlib`.

**What this notebook does**
- Implements Hooke–Jeeves (HJ) and Generalized Pattern Search (GPS).
- Runs both on Ackley, Booth, Branin, Rosenbrock (with coefficient 5), Wheeler, Rastrigin.
- Prints a console table with columns: `function, start, method, x found, f(x found), iterations, time of working`.
- Saves contour plots with optimization trajectories for each function/method.


In [1]:
# Cell 1 — Imports & helpers (minimal)
import numpy as np
import math, time, os
import matplotlib.pyplot as plt

plt.rcParams.update({"figure.dpi": 120})

def norm(x):
    return float(np.linalg.norm(x))

def savefig(path):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d)
    plt.savefig(path, bbox_inches="tight")
    plt.close()

def fmt_vec(v):
    return f"({v[0]:.6f}, {v[1]:.6f})"

def fmt_s(x):
    return f"{x:.6f}"

def print_table(rows, header):
    print(header)
    for r in rows:
        # r = (function, start_tuple, method, x_found_vec2, fval, iterations, time_sec)
        print(f"{r[0]:10s} | {r[1]} | {r[2]:10s} | {fmt_vec(r[3])} | {fmt_s(r[4])} | {r[5]:10d} | {fmt_s(r[6])}")

In [2]:
# Cell 2 — Test functions, known minima, and starting points
def ackley(xy):
    x, y = xy
    term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))
    term2 = -math.exp(0.5*(math.cos(2*math.pi*x)+math.cos(2*math*pi*y)))  # placeholder to show fix

# Correct definition
def ackley(xy):
    x, y = xy
    term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))
    term2 = -math.exp(0.5*(math.cos(2*math.pi*x)+math.cos(2*math.pi*y)))
    return term1 + term2 + math.e + 20.0  # global min at (0,0) with f=0

def booth(xy):
    x, y = xy
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2  # global min at (1,3) with f=0

def branin(xy):
    x, y = xy
    a = 1.0; b = 5.1/(4*math.pi**2); c = 5.0/math.pi; r = 6.0; s = 10.0; t = 1.0/(8*math*pi)  # placeholder to show fix

# Correct definition
def branin(xy):
    x, y = xy
    a = 1.0; b = 5.1/(4*math.pi**2); c = 5.0/math.pi; r = 6.0; s = 10.0; t = 1.0/(8*math.pi)
    return a*(y - b*x*x + c*x - r)**2 + s*(1 - t)*math.cos(x) + s  # one min at (pi, 2.275) with f≈0.397887

def rosenbrock(xy):
    x, y = xy
    return (1.0 - x)**2 + 5.0*(y - x*x)**2  # global min at (1,1) with f=0

def wheeler(xy):
    x, y = xy
    return -math.exp(- (x*y - 1.5)**2 - (y - 1.5)**2)  # global min at (1.5,1.5) with f=-1

def rastrigin(xy):
    x, y = xy
    return 20.0 + (x*x - 10.0*math.cos(2*math.pi*x)) + (y*y - 10.0*math.cos(2*math.pi*y))  # min at (0,0) with f=0

KNOWN_MINIMA = {
    "ackley": [(0.0, 0.0)],
    "booth": [(1.0, 3.0)],
    "branin": [(math.pi, 2.275)],
    "rosenbrock": [(1.0, 1.0)],
    "wheeler": [(1.5, 1.5)],
    "rastrigin": [(0.0, 0.0)]
}

STARTS = {
    "ackley":     np.array([-3.0, -3.0]),
    "booth":      np.array([ 0.0,  0.0]),
    "branin":     np.array([ 2.0,  2.0]),
    "rosenbrock": np.array([-1.5,  2.0]),
    "wheeler":    np.array([ 1.5,  0.5]),
    "rastrigin":  np.array([ 2.5,  2.5])
}

FUNS = {
    "ackley": ackley,
    "booth": booth,
    "branin": branin,
    "rosenbrock": rosenbrock,
    "wheeler": wheeler,
    "rastrigin": rastrigin
}

In [3]:
# Cell 3 — Hooke–Jeeves and GPS (tracking x-history for plotting)
def hooke_jeeves(f, x, alpha, eps, gamma=0.5, max_iter=100000):
    x = np.array(x, float)
    y = f(x)
    n = len(x)
    it = 0
    x_hist = [x.copy()]
    while alpha > eps and it < max_iter:
        improved = False
        x_best = x.copy(); y_best = y
        for i in range(n):
            for sgn in (-1.0, 1.0):
                x_try = x.copy(); x_try[i] += sgn * alpha
                y_try = f(x_try)
                if y_try < y_best:
                    x_best = x_try; y_best = y_try; improved = True
        x, y = x_best, y_best
        x_hist.append(x.copy())
        if not improved:
            alpha *= gamma
        it += 1
    return x, y, it, x_hist

def gps(f, x, alpha, D, eps, gamma=0.5, max_iter=100000):
    x = np.array(x, float)
    y = f(x)
    it = 0
    x_hist = [x.copy()]
    while alpha > eps and it < max_iter:
        improved = False
        for i, d in enumerate(D):
            x_try = x + alpha * d
            y_try = f(x_try)
            if y_try < y:
                x, y = x_try, y_try
                D = [d] + D[:i] + D[i+1:]
                improved = True
                break
        x_hist.append(x.copy())
        if not improved:
            alpha *= gamma
        it += 1
    return x, y, it, x_hist

In [4]:
# Cell 4 — Plotting (contours + trajectory)
def plot_trajectory(fun, x_hist, title, fname, minima, N=200):
    xs = np.array([p[0] for p in x_hist])
    ys = np.array([p[1] for p in x_hist])
    Xs = [xs.min(), xs.max()] + [m[0] for m in minima]
    Ys = [ys.min(), ys.max()] + [m[1] for m in minima]
    xmin, xmax = min(Xs)-2, max(Xs)+2; ymin, ymax = min(Ys)-2, max(Ys)+2
    X = np.linspace(xmin, xmax, N); Y = np.linspace(ymin, ymax, N)
    XX, YY = np.meshgrid(X, Y); ZZ = np.zeros_like(XX)
    for i in range(N):
        for j in range(N):
            ZZ[i,j] = fun(np.array([XX[i,j], YY[i,j]]))
    plt.figure()
    plt.contour(XX, YY, ZZ, levels=30)
    plt.plot(xs, ys, marker='o', linewidth=1)
    plt.scatter([xs[0]], [ys[0]], marker='s', s=40, label='start')
    plt.scatter([xs[-1]], [ys[-1]], marker='*', s=80, label='final')
    for (mx, my) in minima:
        plt.scatter([mx], [my], marker='x', s=50, label='global min')
    plt.legend(); plt.title(title); plt.xlabel('x'); plt.ylabel('y')
    savefig(fname)

In [7]:
# Cell 5 — Run experiments, print results table, save plots
rows = []
alpha0 = 1.0
eps = 1e-6
gamma = 0.5
dirs = [np.array([1.0,0.0]), np.array([-1.0,0.0]), np.array([0.0,1.0]), np.array([0.0,-1.0])]

for name in ['ackley','booth','branin','rosenbrock','wheeler','rastrigin']:
    f = FUNS[name]
    x0 = STARTS[name]

    # Hooke–Jeeves
    t0 = time.time()
    x_hj, y_hj, it_hj, hist_hj = hooke_jeeves(f, x0, alpha0, eps, gamma)
    t1 = time.time()
    plot_trajectory(f, hist_hj, f"Hooke–Jeeves on {name}", f"plot {name} trajectory_HJ.png", KNOWN_MINIMA[name])
    rows.append((name, (float(x0[0]), float(x0[1])), 'Hooke–Jeeves', x_hj, y_hj, it_hj, t1-t0))

    # GPS
    t0 = time.time()
    x_gp, y_gp, it_gp, hist_gp = gps(f, x0, alpha0, dirs.copy(), eps, gamma)
    t1 = time.time()
    plot_trajectory(f, hist_gp, f"GPS on {name}", f"plot {name} trajectory_GPS.png", KNOWN_MINIMA[name])
    rows.append((name, (float(x0[0]), float(x0[1])), 'GPS', x_gp, y_gp, it_gp, t1-t0))

# Console table with required columns
print_table(rows, "function   | start           | method      | x found                 | f(x found) | iterations | time of working")

function   | start           | method      | x found                 | f(x found) | iterations | time of working
ackley     | (-3.0, -3.0) | Hooke–Jeeves | (0.000000, 0.000000) | 0.000000 |         26 | 0.001034
ackley     | (-3.0, -3.0) | GPS        | (0.000000, 0.000000) | 0.000000 |         26 | 0.000509
booth      | (0.0, 0.0) | Hooke–Jeeves | (1.000000, 3.000000) | 0.000000 |         24 | 0.000000
booth      | (0.0, 0.0) | GPS        | (1.000004, 2.999996) | 0.000000 |        100 | 0.000000
branin     | (2.0, 2.0) | Hooke–Jeeves | (3.141592, 2.275000) | 0.397887 |         47 | 0.000000
branin     | (2.0, 2.0) | GPS        | (3.141592, 2.275000) | 0.397887 |         47 | 0.001033
rosenbrock | (-1.5, 2.0) | Hooke–Jeeves | (0.999981, 0.999962) | 0.000000 |        463 | 0.000000
rosenbrock | (-1.5, 2.0) | GPS        | (0.999981, 0.999962) | 0.000000 |        461 | 0.002545
wheeler    | (1.5, 0.5) | Hooke–Jeeves | (1.000000, 1.500000) | -1.000000 |         22 | 0.000000
wheeler    | (1

In [6]:
# Cell 6 — Adaptive GPS (with orthogonalized directions)
def adaptive_gps(f, x, alpha, D, eps, gamma=0.5, max_iter=100000):
    """Adaptive GPS: regenerates orthogonal directions (via Gram–Schmidt) after success."""
    x = np.array(x, float)
    y = f(x)
    it = 0
    x_hist = [x.copy()]
    while alpha > eps and it < max_iter:
        improved = False
        for i, d in enumerate(D):
            x_try = x + alpha * d
            y_try = f(x_try)
            if y_try < y:
                x, y = x_try, y_try
                # Re-orthogonalize directions via Gram–Schmidt
                Q, _ = np.linalg.qr(np.stack(D).T)
                D = [Q[:, j] for j in range(Q.shape[1])]
                improved = True
                break
        x_hist.append(x.copy())
        if not improved:
            alpha *= gamma
        it += 1
    return x, y, it, x_hist

# Run Adaptive GPS on all test functions and print results
rows = []
alpha0 = 1.0; eps = 1e-6; gamma = 0.5
dirs = [np.array([1.0,0.0]), np.array([0.0,1.0])]

for name in ['ackley','booth','branin','rosenbrock','wheeler','rastrigin']:
    f = FUNS[name]; x0 = STARTS[name]
    t0 = time.time()
    x_agp, y_agp, it_agp, hist_agp = adaptive_gps(f, x0, alpha0, dirs.copy(), eps, gamma)
    t1 = time.time()
    rows.append((name, (float(x0[0]), float(x0[1])), 'Adaptive GPS', x_agp, y_agp, it_agp, t1-t0))

# Console output only (no plots)
print_table(rows, "function   | start           | method        | x found                 | f(x found) | iterations | time of working")


function   | start           | method        | x found                 | f(x found) | iterations | time of working
ackley     | (-3.0, -3.0) | Adaptive GPS | (0.000000, 0.000000) | 0.000000 |         26 | 0.014851
booth      | (0.0, 0.0) | Adaptive GPS | (3.000000, 1.500000) | 7.250000 |         25 | 0.001927
branin     | (2.0, 2.0) | Adaptive GPS | (3.125000, 2.500000) | 0.444164 |         23 | 0.000000
rosenbrock | (-1.5, 2.0) | Adaptive GPS | (-1.250000, 2.000000) | 6.019531 |         21 | 0.000000
wheeler    | (1.5, 0.5) | Adaptive GPS | (3.000000, 0.625000) | -0.404037 |         23 | 0.000000
rastrigin  | (2.5, 2.5) | Adaptive GPS | (3.000000, 3.000000) | 18.000000 |         22 | 0.002675
